<a href="https://colab.research.google.com/github/Gennaro-Farina/Advanced-Natural-Language-Processing-with-TensorFlow-2/blob/master/uncertainty_quantification_with_sngp_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Uncertainty-aware Deep Language Learning with BERT-SNGP

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/uncertainty_quantification_with_sngp_bert"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/uncertainty_quantification_with_sngp_bert.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/uncertainty_quantification_with_sngp_bert.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/uncertainty_quantification_with_sngp_bert.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
  <td>
    <a href="https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"><img src="https://www.tensorflow.org/images/hub_logo_32px.png" />See TF Hub model</a>
  </td>
</table>

In the [SNGP tutorial](https://www.tensorflow.org/tutorials/understanding/sngp), you learned how to build SNGP model on top of a deep residual network to improve its ability to quantify its uncertainty. In this tutorial, you will apply SNGP to a natural language understanding (NLU) task by building it on top of a deep BERT encoder to improve deep NLU model's ability in detecting out-of-scope queries. 

Specifically, you will:
* Build BERT-SNGP, a SNGP-augmented [BERT](https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2) model.
* Load the [CLINC Out-of-scope (OOS)](https://www.tensorflow.org/datasets/catalog/clinc_oos) intent detection dataset.
* Train the BERT-SNGP model.
* Evaluate the BERT-SNGP model's performance in uncertainty calibration and out-of-domain detection.

Beyond CLINC OOS, the SNGP model has been applied to large-scale datasets such as [Jigsaw toxicity detection](https://www.tensorflow.org/datasets/catalog/wikipedia_toxicity_subtypes), and to the image datasets such as [CIFAR-100](https://www.tensorflow.org/datasets/catalog/cifar100) and [ImageNet](https://www.tensorflow.org/datasets/catalog/imagenet2012). 
For benchmark results of SNGP and other uncertainty methods, as well as high-quality implementation with end-to-end training / evaluation scripts, you can check out the [Uncertainty Baselines](https://github.com/google/uncertainty-baselines) benchmark.

## Setup

In [ ]:
!pip uninstall -y tensorflow tf-text

In [ ]:
!pip install "tensorflow-text==2.8.*"

In [ ]:
!pip install -U tf-models-official==2.7.0

In [ ]:
import matplotlib.pyplot as plt

import sklearn.metrics
import sklearn.calibration

import tensorflow_hub as hub
import tensorflow_datasets as tfds

import numpy as np
import tensorflow as tf

import official.nlp.modeling.layers as layers
import official.nlp.optimization as optimization

import sklearn
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

This tutorial needs the GPU to run efficiently. Check if the GPU is available. 

In [ ]:
tf.__version__

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
gpus

In [ ]:
assert gpus, """
  No GPU(s) found! This tutorial will take many hours to run without a GPU.

  You may hit this error if the installed tensorflow package is not
  compatible with the CUDA and CUDNN versions."""

First implement a standard BERT classifier following the [classify text with BERT](https://www.tensorflow.org/tutorials/text/classify_text_with_bert) tutorial. We will use the [BERT-base](https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3) encoder, and the built-in [`ClassificationHead`](https://github.com/tensorflow/models/blob/master/official/nlp/modeling/layers/cls_head.py) as the classifier.

In [ ]:
#@title Standard BERT model

PREPROCESS_HANDLE = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
MODEL_HANDLE = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3'

class BertClassifier(tf.keras.Model):
  def __init__(self, 
               num_classes=150, inner_dim=768, dropout_rate=0.1,
               **classifier_kwargs):
    
    super().__init__()
    self.classifier_kwargs = classifier_kwargs

    # Initiate the BERT encoder components.
    self.bert_preprocessor = hub.KerasLayer(PREPROCESS_HANDLE, name='preprocessing')
    self.bert_hidden_layer = hub.KerasLayer(MODEL_HANDLE, trainable=True, name='bert_encoder')

    # Defines the encoder and classification layers.
    self.bert_encoder = self.make_bert_encoder()
    self.classifier = self.make_classification_head(num_classes, inner_dim, dropout_rate)

  def make_bert_encoder(self):
    text_inputs = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    encoder_inputs = self.bert_preprocessor(text_inputs)
    encoder_outputs = self.bert_hidden_layer(encoder_inputs)
    return tf.keras.Model(text_inputs, encoder_outputs)

  def make_classification_head(self, num_classes, inner_dim, dropout_rate):
    return layers.ClassificationHead(
        num_classes=num_classes, 
        inner_dim=inner_dim,
        dropout_rate=dropout_rate,
        **self.classifier_kwargs)

  def call(self, inputs, **kwargs):
    encoder_outputs = self.bert_encoder(inputs)
    classifier_inputs = encoder_outputs['sequence_output']
    return self.classifier(classifier_inputs, **kwargs)


### Build SNGP model

To implement a BERT-SNGP model, you only need to replace the `ClassificationHead` with the built-in [`GaussianProcessClassificationHead`](https://github.com/tensorflow/models/blob/master/official/nlp/modeling/layers/cls_head.py). Spectral normalization is already pre-packaged into this classification head. Like in the [SNGP tutorial](https://www.tensorflow.org/tutorials/uncertainty/sngp), add a covariance reset callback to the model, so the model automatically reset the covariance estimator at the beginning of a new epoch to avoid counting the same data twice.

In [ ]:
class ResetCovarianceCallback(tf.keras.callbacks.Callback):

  def on_epoch_begin(self, epoch, logs=None):
    """Resets covariance matrix at the beginning of the epoch."""
    if epoch > 0:
      self.model.classifier.reset_covariance_matrix()

In [ ]:
class SNGPBertClassifier(BertClassifier):

  def make_classification_head(self, num_classes, inner_dim, dropout_rate):
    return layers.GaussianProcessClassificationHead(
        num_classes=num_classes, 
        inner_dim=inner_dim,
        dropout_rate=dropout_rate,
        gp_cov_momentum=-1,
        temperature=30.,
        **self.classifier_kwargs)

  def fit(self, *args, **kwargs):
    """Adds ResetCovarianceCallback to model callbacks."""
    kwargs['callbacks'] = list(kwargs.get('callbacks', []))
    kwargs['callbacks'].append(ResetCovarianceCallback())

    return super().fit(*args, **kwargs)

Note: The `GaussianProcessClassificationHead` takes a new argument `temperature`. It corresponds to the $\lambda$ parameter in the __mean-field approximation__ introduced in the [SNGP tutorial](https://www.tensorflow.org/tutorials/understanding/sngp). In practice, this value is usually treated as a hyperparameter, and is finetuned to optimize the model's calibration performance.

### Load CLINC OOS dataset

Now load the [CLINC OOS](https://www.tensorflow.org/datasets/catalog/clinc_oos) intent detection dataset. This dataset contains 15000 user's spoken queries collected over 150 intent classes, it also contains 1000 out-of-domain (OOD) sentences that are not covered by any of the known classes.

In [ ]:
(clinc_train, clinc_test, clinc_test_oos, clinc_validation), ds_info = tfds.load(
    'clinc_oos', split=['train', 'test', 'test_oos', 'validation'], with_info=True, batch_size=-1)

Make the train and test data.

In [ ]:
train_examples = clinc_train['text']
train_labels = clinc_train['intent']

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(
#     train_examples.numpy(), train_labels.numpy(), test_size=0.20, random_state=42
# )

In [ ]:
val_examples = clinc_validation['text']
val_labels = clinc_validation['intent']

ind_val_data = (val_examples, val_labels)

In [ ]:
test_examples = clinc_test['text']
test_labels = clinc_test['intent']

# Makes the in-domain (IND) evaluation data.
ind_test_data = (test_examples, test_labels)

print('num intents {} :\n {}'.format(len(np.unique(clinc_test['intent_name'])),
                       clinc_test['intent_name']))

In [ ]:
ind_test_data

Create a OOD evaluation dataset. For this, combine the in-domain test data `clinc_test` and the out-of-domain data `clinc_test_oos`. We will also assign label 0 to the in-domain examples, and label 1 to the out-of-domain examples. 

In [ ]:
test_data_size = ds_info.splits['test'].num_examples
oos_data_size = ds_info.splits['test_oos'].num_examples

# Combines the in-domain and out-of-domain test examples.
oos_texts = tf.concat([clinc_test['text'], clinc_test_oos['text']], axis=0)
oos_labels = tf.constant([0] * test_data_size + [1] * oos_data_size)

# Converts into a TF dataset.
ood_eval_dataset = tf.data.Dataset.from_tensor_slices(
    {"text": oos_texts, "label": oos_labels})

ood_eval_dataset_inscope = tf.data.Dataset.from_tensor_slices(
    {"text": oos_texts[:test_data_size], "label": oos_labels[:test_data_size]})

ood_eval_dataset_oos = tf.data.Dataset.from_tensor_slices(
    {"text": oos_texts[-oos_data_size:], "label": oos_labels[-oos_data_size:]})

### Train and evaluate

First set up the basic training configurations.

In [ ]:
TRAIN_EPOCHS = 10
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 256

In [ ]:
#@title

def bert_optimizer(learning_rate, 
                   batch_size=TRAIN_BATCH_SIZE, epochs=TRAIN_EPOCHS, 
                   warmup_rate=0.1):
  """Creates an AdamWeightDecay optimizer with learning rate schedule."""
  train_data_size = ds_info.splits['train'].num_examples
  
  steps_per_epoch = int(train_data_size / batch_size)
  num_train_steps = steps_per_epoch * epochs
  num_warmup_steps = int(warmup_rate * num_train_steps)  

  # Creates learning schedule.
  lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
      initial_learning_rate=learning_rate,
      decay_steps=num_train_steps,
      end_learning_rate=0.0)  
  
  return optimization.AdamWeightDecay(
      learning_rate=lr_schedule,
      weight_decay_rate=0.01,
      epsilon=1e-6,
      exclude_from_weight_decay=['LayerNorm', 'layer_norm', 'bias'])

In [ ]:
optimizer = bert_optimizer(learning_rate=1e-4)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = tf.metrics.SparseCategoricalAccuracy()

In [ ]:
fit_configs = dict(batch_size=TRAIN_BATCH_SIZE,
                   epochs=TRAIN_EPOCHS,
                   validation_batch_size=EVAL_BATCH_SIZE, 
                   validation_data=ind_test_data)

In [ ]:
bert_model = BertClassifier()
bert_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
bert_model.fit(train_examples, train_labels, **fit_configs)

In [ ]:
bert_out = bert_model.evaluate(test_examples, test_labels)

In [ ]:
bert_out

In [ ]:
bert_predicted_labels = bert_model.predict(test_examples)

In [ ]:
bert_f1_macro_score = sklearn.metrics.f1_score(test_labels,
                         tf.math.argmax(bert_predicted_labels, axis=1),
                         average='macro'
                        )

bert_cm = sklearn.metrics.confusion_matrix(test_labels,
                                 tf.math.argmax(bert_predicted_labels, axis=1)
                                 )

test_label = clinc_test['intent']
test_name  = clinc_test['intent_name']
labels_names_dict = {i: t for i, t in zip(test_label.numpy(), test_name.numpy())}
labels_names_dict_sorted = {i:labels_names_dict[i] for i in sorted(labels_names_dict)}

df_cm = pd.DataFrame(bert_cm,
                     index = [i for _, i in labels_names_dict_sorted.items()],
                     columns = [i for _, i in labels_names_dict_sorted.items()])
plt.figure(figsize = (100,70))
sn.heatmap(df_cm, annot=True)
plt.savefig('confusion_matrix_BERT_HEAD.png', dpi=100)
plt.show()

In [ ]:
sngp_model = SNGPBertClassifier()
sngp_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
sngp_model.fit(train_examples, train_labels, **fit_configs)

In [ ]:
train_labels

In [ ]:
sngp_out = sngp_model.evaluate(test_examples, test_labels)

In [ ]:
sngp_out

In [ ]:
predicted_labels = sngp_model.predict(test_examples)

In [ ]:
# for test_label in test_labels:
#   print(test_label)

In [ ]:
tf.math.argmax(predicted_labels, axis=1)

In [ ]:
f1_macro_score = sklearn.metrics.f1_score(test_labels,
                         tf.math.argmax(predicted_labels, axis=1),
                         average='macro'
                        )

cm = sklearn.metrics.confusion_matrix(test_labels,
                                 tf.math.argmax(predicted_labels, axis=1)
                                 )

test_label = clinc_test['intent']
test_name  = clinc_test['intent_name']
labels_names_dict = {i: t for i, t in zip(test_label.numpy(), test_name.numpy())}
labels_names_dict_sorted = {i:labels_names_dict[i] for i in sorted(labels_names_dict)}

df_cm = pd.DataFrame(cm, index = [i for _, i in labels_names_dict_sorted.items()],
                  columns = [i for _, i in labels_names_dict_sorted.items()])
plt.figure(figsize = (100,70))
sn.heatmap(df_cm, annot=True)
plt.savefig('confusion_matrix_SNGP_HEAD.png', dpi=100)
plt.show()

In [ ]:
ds_info

### Evaluate OOD performance

Evaluate how well the model can detect the unfamiliar out-of-domain queries. For rigorous evaluation, use the OOD evaluation dataset `ood_eval_dataset` built earlier.

In [ ]:
#@title

def oos_predict(model, ood_eval_dataset, **model_kwargs):
  oos_labels = []
  oos_probs = []

  ood_eval_dataset = ood_eval_dataset.batch(EVAL_BATCH_SIZE)
  for oos_batch in ood_eval_dataset:
    oos_text_batch = oos_batch["text"]
    oos_label_batch = oos_batch["label"] 

    pred_logits = model(oos_text_batch, **model_kwargs)
    pred_probs_all = tf.nn.softmax(pred_logits, axis=-1)
    pred_probs = tf.reduce_max(pred_probs_all, axis=-1)

    oos_labels.append(oos_label_batch)
    oos_probs.append(pred_probs)

  oos_probs = tf.concat(oos_probs, axis=0)
  oos_labels = tf.concat(oos_labels, axis=0) 

  return oos_probs, oos_labels

Computes the OOD probabilities as $1 - p(x)$, where $p(x)=softmax(logit(x))$ is the predictive probability.

In [ ]:
sngp_probs, ood_labels = oos_predict(sngp_model, ood_eval_dataset)

In [ ]:
def oos_predict2(model, ood_eval_dataset, **model_kwargs):
  oos_labels = []
  oos_probs = []

  ood_eval_dataset = ood_eval_dataset.batch(EVAL_BATCH_SIZE)
  for oos_batch in ood_eval_dataset:
    oos_text_batch = oos_batch["text"]
    # print('oos_batch["label"]: ', oos_batch["label"] )
    oos_label_batch = oos_batch["label"] 

    pred_logits = model(oos_text_batch, **model_kwargs)
    # print(pred_logits.shape)
    pred_probs_all = tf.nn.softmax(pred_logits, axis=-1)

    pred_probs = tf.reduce_max(pred_probs_all, axis=-1)

    # print('sngp_probs: ', pred_probs_all)
    for pred, text, lbl, prob in zip(pred_logits, oos_text_batch, oos_label_batch, pred_probs):  
      # print('pred: {}\n prob_sum: {} pred_lbl: {} text: {}\n lbl: {}\n\n\n '.format(pred,
      #                                                                               np.sum(pred_probs),
      #                                                                               np.argmax(pred_probs),
      #                                                                               text,
      #                                                                               lbl))
      
      print('prob: {:2.2f} argmax: {}  text: {} '.format( np.max(prob), labels_names_dict[np.argmax(pred)], text, ))

    oos_labels.append(oos_label_batch)
    oos_probs.append(pred_probs)

  oos_probs = tf.concat(oos_probs, axis=0)
  oos_labels = tf.concat(oos_labels, axis=0) 

  return oos_probs, oos_labels

# print('NO SNGP MODEL ***************************')
# print('inscope_results: ')
# sngp_probs, ood_labels = oos_predict2(bert_model, ood_eval_dataset_inscope)
# # print('ood_labels: ', ood_labels)

# print('oos_results: ')
# sngp_probs, ood_labels = oos_predict2(bert_model, ood_eval_dataset_oos)
# # print('ood_labels: ', ood_labels)

print('\n\n\n')
print('SNGP MODEL ***************************')
# print('inscope_results: ')
# sngp_probs, ood_labels = oos_predict2(sngp_model, ood_eval_dataset_inscope)
# # print('ood_labels: ', ood_labels)

print('oos_results: ')
sngp_probs, ood_labels = oos_predict2(sngp_model, ood_eval_dataset_oos)
# print('ood_labels: ', ood_labels)



In [ ]:
for oos_batch in ood_eval_dataset:
  oos_text_batch = oos_batch["text"]
  oos_label_batch = oos_batch["label"] 
  print(oos_text_batch, oos_label_batch)

In [ ]:
sngp_model.save('sngp_model_10epochs')

In [ ]:
sngp_model2 = tf.keras.models.load_model('sngp_model_10epochs', custom_objects={'AdamWeightDecay':optimizer})

In [ ]:
sngp_probs, ood_labels = oos_predict2(sngp_model2, ood_eval_dataset_oos)
# print('ood_labels: ', ood_labels)

In [ ]:
! zip -r sngp_model_10_epochs sngp_model_10epochs/*

In [ ]:
ood_probs = 1 - sngp_probs

Now evaluate how well the model's uncertainty score `ood_probs` predicts the out-of-domain label. First compute the Area under precision-recall curve (AUPRC) for OOD probability v.s. OOD detection accuracy.

In [ ]:
precision, recall, _ = sklearn.metrics.precision_recall_curve(ood_labels, ood_probs)

In [ ]:
auprc = sklearn.metrics.auc(recall, precision)
print(f'SNGP AUPRC: {auprc:.4f}')

This matches the SNGP performance reported at the CLINC OOS benchmark under the [Uncertainty Baselines](https://github.com/google/uncertainty-baselines).

Next, examine the model's quality in [uncertainty calibration](https://scikit-learn.org/stable/modules/calibration.html), i.e., whether the model's predictive probability corresponds to its predictive accuracy. A well-calibrated model is considered trust-worthy, since, for example, its predictive probability $p(x)=0.8$ means that the model is correct 80% of the time.

In [ ]:
prob_true, prob_pred = sklearn.calibration.calibration_curve(
    ood_labels, ood_probs, n_bins=10, strategy='quantile')

In [ ]:
plt.plot(prob_pred, prob_true)

plt.plot([0., 1.], [0., 1.], c='k', linestyle="--")
plt.xlabel('Predictive Probability')
plt.ylabel('Predictive Accuracy')
plt.title('Calibration Plots, SNGP')

plt.show()

## Resources and further reading

* See the [SNGP tutorial](https://www.tensorflow.org/tutorials/understanding/sngp) for a detailed walkthrough of implementing SNGP from scratch. 
* See [Uncertainty Baselines](https://github.com/google/uncertainty-baselines)  for the implementation of SNGP model (and many other uncertainty methods) on a wide variety of benchmark datasets (e.g., [CIFAR](https://www.tensorflow.org/datasets/catalog/cifar100), [ImageNet](https://www.tensorflow.org/datasets/catalog/imagenet2012), [Jigsaw toxicity detection](https://www.tensorflow.org/datasets/catalog/wikipedia_toxicity_subtypes), etc).
* For a deeper understanding of the SNGP method, check out the paper [Simple and Principled Uncertainty Estimation with Deterministic Deep Learning via Distance Awareness](https://arxiv.org/abs/2006.10108).
